$0.79425$。

https://zhuanlan.zhihu.com/p/30538352

Kaggle泰坦尼克号生存模型——250个特征量的融合模型，排名8%。

要建立新的特征类目：

1、Fare Category
2、Pclass Fare Category
3、Family Size Category
4、Age Group Category
5、Name Length Category

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 开始训练模型

train_data_org = pd.read_csv('../input/train.csv')
test_data_org = pd.read_csv('../input/test.csv')

In [2]:
# 弃掉不需要的列
def drop_col_not_req(df, cols):
    df.drop(cols, axis=1, inplace=True)

In [3]:
# 建立 Fare Category
def fare_category(fare):
    if fare <= 4:
        return 'Very_Low_Fare'
    elif fare <= 10:
        return 'Low_Fare'
    elif fare <= 30:
        return 'Med_Fare'
    elif fare <= 45:
        return 'High_Fare'
    else:
        return 'Very_High_Fare'

In [4]:
# 建立 PClass Fare Category
def pclass_fare_category(df, Pclass_1_mean_fare, Pclass_2_mean_fare,
                         Pclass_3_mean_fare):
    if df['Pclass'] == 1:
        if df['Fare'] <= Pclass_1_mean_fare:
            return 'Pclass_1_Low_Fare'
        else:
            return 'Pclass_1_High_Fare'
    elif df['Pclass'] == 2:
        if df['Fare'] <= Pclass_2_mean_fare:
            return 'Pclass_2_Low_Fare'
        else:
            return 'Pclass_2_High_Fare'
    elif df['Pclass'] == 3:
        if df['Fare'] <= Pclass_3_mean_fare:
            return 'Pclass_3_Low_Fare'
        else:
            return 'Pclass_3_High_Fare'

In [5]:
# 建立Family Size Category
def family_size_category(family_size):
    if family_size <= 1:
        return 'Single'
    elif family_size <= 3:
        return 'Small_Family'
    else:
        return 'Large_Family'

In [6]:
# 建立Age_Group_category
def age_group_category(age):
    if age <= 1:
        return 'Baby'
    elif age <= 4:
        return 'Toddler'
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teenager'
    elif age <= 30:
        return 'Adult'
    elif age <= 50:
        return 'Middle_Aged'
    elif age < 60:
        return 'Senior_Citizen'
    else:
        return 'Old'

In [7]:
# 建立 Name_Length_Category
def name_length_category(name_len):
    if name_len <= 19:
        return 'Very_Short_Name'
    elif name_len <= 28:
        return 'Short_Name'
    elif name_len <= 45:
        return 'Medium_Name'
    else:
        return 'Long_Name'

In [8]:
# 填充NaN值
# 使用GradientBoostingRegressor和LinearRegression来填充Age值
def fill_missing_age(missing_age_train, missing_age_test):
    missing_age_X_train = missing_age_train.drop(['Age'], axis=1)
    missing_age_Y_train = missing_age_train['Age']
    missing_age_X_test = missing_age_test.drop(['Age'], axis=1)

    gbm_reg = ensemble.GradientBoostingRegressor(random_state=42)
    gbm_reg_param_grid = {'n_estimators': [2000], 'max_depth': [3],
                          'learning_rate': [0.01], 'max_features': [3]}
    gbm_reg_grid = model_selection.GridSearchCV(gbm_reg, gbm_reg_param_grid,
                                                cv=10, n_jobs=25, verbose=1,
                                                scoring='neg_mean_squared_error')
    gbm_reg_grid.fit(missing_age_X_train, missing_age_Y_train)
    print('Age feature Best GB Params:' + str(gbm_reg_grid.best_params_))
    print('Age feature Best GB Score:' + str(gbm_reg_grid.best_score_))
    print('GB Train Error for "Age" Feature Regressor:' + str(
        gbm_reg_grid.score(missing_age_X_train, missing_age_Y_train)))
    missing_age_test['Age_GB'] = gbm_reg_grid.predict(missing_age_X_test)
    print(missing_age_test['Age_GB'][:4])

    lrf_reg = LinearRegression()
    lrf_reg_param_grid = {'fit_intercept': [True], 'normalize': [True]}
    lrf_reg_grid = model_selection.GridSearchCV(lrf_reg, lrf_reg_param_grid,
                                                cv=10, n_jobs=25, verbose=1,
                                                scoring='neg_mean_squared_error')
    lrf_reg_grid.fit(missing_age_X_train, missing_age_Y_train)
    print('Age feature Best LR Params:' + str(lrf_reg_grid.best_params_))
    print('Age feature Best LR Score:' + str(lrf_reg_grid.best_score_))
    print('LR Train Error for "Age" Feature Regressor' + str(
        lrf_reg_grid.score(missing_age_X_train, missing_age_Y_train)))

    missing_age_test['Age_LRF'] = lrf_reg_grid.predict(missing_age_X_test)
    print(missing_age_test['Age_LRF'][:4])

    print('shape1', missing_age_test['Age'].shape, missing_age_test[['Age_GB', 'Age_LRF']].mode(axis=1).shape)
    # missing_age_test['Age'] = missing_age_test[['Age_GB','Age_LRF']].mode(axis=1)
    missing_age_test['Age'] = np.mean([missing_age_test['Age_GB'], missing_age_test['Age_LRF']])
    print(missing_age_test['Age'][:4])
    drop_col_not_req(missing_age_test, ['Age_GB', 'Age_LRF'])

    return missing_age_test

In [9]:
# 筛选重要特征
def get_top_n_features(titanic_train_data_X, titanic_train_data_Y, top_n_features):
    # 随机森林
    rf_est = RandomForestClassifier(random_state=42)
    rf_param_grid = {'n_estimators': [500], 'min_samples_split': [2, 3], 'max_depth': [20]}
    rf_grid = model_selection.GridSearchCV(rf_est, rf_param_grid, n_jobs=25, cv=10, verbose=1)
    rf_grid.fit(titanic_train_data_X, titanic_train_data_Y)

    print('Top N Features Best RF Params:' + str(rf_grid.best_params_))
    print('Top N Features Best RF Score:' + str(rf_grid.best_score_))
    print('Top N Features RF Train Error:' + str(rf_grid.score(titanic_train_data_X, titanic_train_data_Y)))

    feature_imp_sorted_rf = pd.DataFrame({'feature': list(titanic_train_data_X),
                                          'importance': rf_grid.best_estimator_.feature_importances_}).sort_values(
        'importance', ascending=False)
    features_top_n_rf = feature_imp_sorted_rf.head(top_n_features)['feature']
    print('Sample 25 Features from RF Classifier')
    print(str(features_top_n_rf[:25]))

    # AdaBoost
    ada_est = ensemble.AdaBoostClassifier(random_state=42)
    ada_param_grid = {'n_estimators': [500], 'learning_rate': [0.5, 0.6]}
    ada_grid = model_selection.GridSearchCV(ada_est, ada_param_grid, n_jobs=25, cv=10, verbose=1)
    ada_grid.fit(titanic_train_data_X, titanic_train_data_Y)

    print('Top N Features Best Ada Params:' + str(ada_grid.best_params_))
    print('Top N Features Best Ada Score:' + str(ada_grid.best_score_))
    print('Top N Features Ada Train Error:' + str(ada_grid.score(titanic_train_data_X, titanic_train_data_Y)))

    feature_imp_sorted_ada = pd.DataFrame({'feature': list(titanic_train_data_X),
                                           'importance': ada_grid.best_estimator_.feature_importances_}).sort_values(
        'importance', ascending=False)
    features_top_n_ada = feature_imp_sorted_ada.head(top_n_features)['feature']
    print('Sample 25 Feature from Ada Classifier:')
    print(str(features_top_n_ada[:25]))

    # ExtraTree
    et_est = ensemble.ExtraTreesClassifier(random_state=42)
    et_param_grid = {'n_estimators': [500], 'min_samples_split': [3, 4], 'max_depth': [15]}
    et_grid = model_selection.GridSearchCV(et_est, et_param_grid, n_jobs=25, cv=10, verbose=1)
    et_grid.fit(titanic_train_data_X, titanic_train_data_Y)

    print('Top N Features Best ET Params:' + str(et_grid.best_params_))
    print('Top N Features Best ET Score:' + str(et_grid.best_score_))
    print('Top N Features ET Train Error:' + str(et_grid.score(titanic_train_data_X, titanic_train_data_Y)))

    feature_imp_sorted_et = pd.DataFrame({'feature': list(titanic_train_data_X),
                                          'importance': et_grid.best_estimator_.feature_importances_}).sort_values(
        'importance', ascending=False)
    features_top_n_et = feature_imp_sorted_et.head(top_n_features)['feature']
    print('Sample 25 Features from ET Classifier:')
    print(str(features_top_n_et[:25]))

    # 融合以上三个模型
    features_top_n = pd.concat([features_top_n_rf, features_top_n_ada, features_top_n_et],
                               ignore_index=True).drop_duplicates()

    return features_top_n

In [10]:
# 将训练集和测试集合并

test_data_org['Survived'] = 0
combined_train_test = train_data_org.append(test_data_org, sort=False)

In [11]:
import os
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn import ensemble
from sklearn import model_selection

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [12]:
# 特征工程

# 1. Embarked
# Embarkde中有两个NaN
if combined_train_test['Embarked'].isnull().sum() != 0:
    combined_train_test['Embarked'].fillna(
        combined_train_test['Embarked'].mode().iloc[0], inplace=True)

emb_dummies_df = pd.get_dummies(
    combined_train_test['Embarked'],
    prefix=combined_train_test[['Embarked']].columns[0])
combined_train_test = pd.concat([combined_train_test, emb_dummies_df], axis=1)

# 2. Sex
sex_dummies_df = pd.get_dummies(
    combined_train_test['Sex'], prefix=combined_train_test[['Sex']].columns[0])
combined_train_test = pd.concat([combined_train_test, sex_dummies_df], axis=1)

# 3. Name
# Title
combined_train_test['Title'] = combined_train_test['Name'].str.extract(
    ' ([A-Za-z]+)\.', expand=False)
# print(combined_train_test['Title'].groupby(by=combined_train_test['Title']).count().sort_values(ascending=False))

title_Dict = {}
title_Dict.update(
    dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
title_Dict.update(
    dict.fromkeys(['Jonkheer', 'Don', 'Sir', 'the Countess', 'Dona', 'Lady'],
                  'Royalty'))
title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
title_Dict.update(dict.fromkeys(['Mr'], 'Mr'))
title_Dict.update(dict.fromkeys(['Master'], 'Master'))

combined_train_test['Title'] = combined_train_test['Title'].map(title_Dict)
# print(combined_train_test['Title'].groupby(by=combined_train_test['Title']).count().sort_values(ascending=False))

title_dummies_df = pd.get_dummies(
    combined_train_test['Title'],
    prefix=combined_train_test[['Title']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, title_dummies_df], axis=1)
# print(combined_train_test)

# Name Length
combined_train_test['Name_Length'] = combined_train_test['Name'].str.len()
combined_train_test['Name_length_Category'] = combined_train_test[
    'Name_Length'].map(name_length_category)
# print(combined_train_test['Name_length_Category'].groupby(by=combined_train_test['Name_length_Category']).count().sort_values(ascending=False))

le_name = LabelEncoder()
le_name.fit(
    np.array(['Very_Short_Name', 'Short_Name', 'Medium_Name', 'Long_Name']))
combined_train_test['Name_length_Category'] = le_name.transform(
    combined_train_test['Name_length_Category'])

# print(combined_train_test[['Name_length_Category','Survived']].corr())
name_length_dummies_df = pd.get_dummies(
    combined_train_test['Name_length_Category'],
    prefix=combined_train_test[['Name_length_Category']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, name_length_dummies_df], axis=1)
# print(combined_train_test)

# First Name
combined_train_test['First_Name'] = combined_train_test['Name'].str.extract(
    '^(.+?),')
first_name_dummies_df = pd.get_dummies(
    combined_train_test['First_Name'],
    prefix=combined_train_test[['First_Name']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, first_name_dummies_df], axis=1)
# print(combined_train_test)

# Last_Name
combined_train_test['Last_Name'] = combined_train_test['Name'].str.split('\.')
combined_train_test['Last_Name'] = combined_train_test['Last_Name'].str.strip(
    '\([^)]*\)')
combined_train_test['Last_Name'].fillna(
    combined_train_test['Name'].str.split('\.').str[1])
# print(combined_train_test['Last_Name'].groupby(by = combined_train_test['Last_Name']).count().sort_values(ascending = False)[:5])
last_name_dummies_df = pd.get_dummies(
    combined_train_test['Last_Name'],
    prefix=combined_train_test[['Last_Name']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, last_name_dummies_df], axis=1)

# print(combined_train_test)

# Original_Name
combined_train_test['Original_Name'] = combined_train_test['Name'].str.split(
    '\((.*?)\)').str[1].str.strip('\"').str.strip()
# print(combined_train_test['Original_Name'].groupby(by = combined_train_test['Original_Name']).count().sort_values(ascending = False)[:5])
original_name_dummies_df = pd.get_dummies(
    combined_train_test['Original_Name'],
    prefix=combined_train_test[['Original_Name']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, original_name_dummies_df], axis=1)

# 4. Fare
# 填充NaN
if combined_train_test['Fare'].isnull().sum() != 0:
    combined_train_test['Fare'] = combined_train_test[['Fare']].fillna(
        combined_train_test.groupby('Pclass').transform('mean'))

# 将多人船票的价格平均到每人
combined_train_test['Group_Ticket'] = combined_train_test['Fare'].groupby(
    by=combined_train_test['Ticket']).transform('count')
combined_train_test['Fare'] = combined_train_test[
    'Fare'] / combined_train_test['Group_Ticket']
combined_train_test.drop(['Group_Ticket'], axis=1, inplace=True)

# 去除Fare为0的项
if sum(n == 0 for n in combined_train_test.Fare.values.flatten()) > 0:
    combined_train_test.loc[combined_train_test.Fare == 0, 'Fare'] = np.nan
    combined_train_test['Fare'] = combined_train_test[['Fare']].fillna(
        combined_train_test.groupby('Pclass').transform('mean'))
# print(combined_train_test['Fare'].describe())
# 建立Fare Category
combined_train_test['Fare_Category'] = combined_train_test['Fare'].map(
    fare_category)
le_fare = LabelEncoder()
le_fare.fit(
    np.array([
        'Very_Low_Fare', 'Low_Fare', 'Med_Fare', 'High_Fare', 'Very_High_Fare'
    ]))
combined_train_test['Fare_Category'] = le_fare.transform(
    combined_train_test['Fare_Category'])

fare_cat_dummies_df = pd.get_dummies(
    combined_train_test['Fare_Category'],
    prefix=combined_train_test[['Fare_Category']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, fare_cat_dummies_df], axis=1)
# print(combined_train_test['Fare_Category'].groupby(by = combined_train_test['Fare_Category']).count().sort_values(ascending = False))

# 5. Pclass
# print(combined_train_test['Fare'].groupby(by = combined_train_test['Pclass']).mean())
Pclass_1_mean_fare = combined_train_test['Fare'].groupby(
    by=combined_train_test['Pclass']).mean().get([1]).values[0]
Pclass_2_mean_fare = combined_train_test['Fare'].groupby(
    by=combined_train_test['Pclass']).mean().get([2]).values[0]
Pclass_3_mean_fare = combined_train_test['Fare'].groupby(
    by=combined_train_test['Pclass']).mean().get([3]).values[0]
# 建立Pclass_Fare Category
combined_train_test['Pclass_Fare_Category'] = combined_train_test.apply(
    pclass_fare_category,
    args=(Pclass_1_mean_fare, Pclass_2_mean_fare, Pclass_3_mean_fare),
    axis=1)
# print(combined_train_test['Pclass_Fare_Category'].groupby(by = combined_train_test['Pclass_Fare_Category']).count().sort_values(ascending = False))
p_fare = LabelEncoder()
p_fare.fit(
    np.array([
        'Pclass_1_Low_Fare', 'Pclass_1_High_Fare', 'Pclass_2_Low_Fare',
        'Pclass_2_High_Fare', 'Pclass_3_Low_Fare', 'Pclass_3_High_Fare'
    ]))
combined_train_test['Pclass_Fare_Category'] = p_fare.transform(
    combined_train_test['Pclass_Fare_Category'])

# 6. Parch and SibSp

combined_train_test[
    'Family_Size'] = combined_train_test['Parch'] + combined_train_test['SibSp'] + 1
combined_train_test['Family_Size_Category'] = combined_train_test[
    'Family_Size'].map(family_size_category)

le_family = LabelEncoder()
le_family.fit(np.array(['Single', 'Small_Family', 'Large_Family']))
combined_train_test['Family_Size_Category'] = le_family.transform(
    combined_train_test['Family_Size_Category'])

fam_size_cat_dummies_df = pd.get_dummies(
    combined_train_test['Family_Size_Category'],
    prefix=combined_train_test[['Family_Size_Category']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, fam_size_cat_dummies_df], axis=1)
# print(combined_train_test)

# 7. Age
# 填充Age中的NaN值

combined_train_test['Age_Null'] = combined_train_test['Age'].apply(
    lambda x: 1 if pd.notnull(x) else 0)

missing_age_df = pd.DataFrame(combined_train_test[[
    'Age', 'Parch', 'Sex', 'SibSp', 'Family_Size', 'Family_Size_Category',
    'Title', 'Fare', 'Fare_Category', 'Pclass', 'Embarked'
]])
missing_age_df = pd.get_dummies(
    missing_age_df,
    columns=[
        'Title', 'Family_Size_Category', 'Fare_Category', 'Sex', 'Pclass',
        'Embarked'
    ])
missing_age_train = missing_age_df[missing_age_df['Age'].notnull()]
missing_age_test = missing_age_df[missing_age_df['Age'].isnull()]
combined_train_test.loc[(
    combined_train_test.Age.isnull()), 'Age'] = fill_missing_age(
        missing_age_train, missing_age_test)
# print(combined_train_test.describe())

# 检查是否有异常值
if sum(n < 0 for n in combined_train_test.Age.values.flatten()) > 0:
    combined_train_test.loc[combined_train_test.Age < 0, 'Age'] = np.nan
    combined_train_test['Age'] = combined_train_test[['Age']].fillna(
        combined_train_test.groupby('Title').transform('mean'))
# print(combined_train_test['Age'].groupby(by=combined_train_test['Title']).mean().sort_values(ascending=True))

# 建立Age_Category
combined_train_test['Age_Category'] = combined_train_test['Age'].map(
    age_group_category)
le_age = LabelEncoder()
le_age.fit(
    np.array([
        'Baby', 'Toddler', 'Child', 'Teenager', 'Adult', 'Middle_Aged',
        'Senior_Citizen', 'Old'
    ]))
combined_train_test['Age_Category'] = le_age.transform(
    combined_train_test['Age_Category'])
age_cat_dummies_df = pd.get_dummies(
    combined_train_test['Age_Category'],
    prefix=combined_train_test[['Age_Category']].columns[0])
combined_train_test = pd.concat(
    [combined_train_test, age_cat_dummies_df], axis=1)

# 8. Ticket
combined_train_test['Ticket_Letter'] = combined_train_test[
    'Ticket'].str.split().str[0]
combined_train_test['Ticket_Letter'] = combined_train_test[
    'Ticket_Letter'].apply(lambda x: np.nan if x.isnumeric() else x)
combined_train_test['Ticket_Number'] = combined_train_test['Ticket'].apply(
    lambda x: pd.to_numeric(x, errors='coerce'))
combined_train_test['Ticket_Number'].fillna(0, inplace=True)
combined_train_test = pd.get_dummies(
    combined_train_test, columns=['Ticket', 'Ticket_Letter'])
# print(combined_train_test.shape)

# 9. Cabin
combined_train_test['Cabin_Letter'] = combined_train_test['Cabin'].apply(
    lambda x: str(x)[0] if pd.notnull(x) else x)
combined_train_test = pd.get_dummies(
    combined_train_test, columns=['Cabin', 'Cabin_Letter'])
# print(combined_train_test.shape)

# 10. 将Age和Fare正则化
scale_age_fare = preprocessing.StandardScaler().fit(
    combined_train_test[['Age', 'Fare']])
combined_train_test[['Age', 'Fare']] = scale_age_fare.transform(
    combined_train_test[['Age', 'Fare']])

# 11. 弃掉无用列
combined_train_test.drop(
    [
        'Name', 'PassengerId', 'Embarked', 'Sex', 'Title', 'Fare_Category',
        'Family_Size_Category', 'Age_Category', 'First_Name', 'Last_Name',
        'Original_Name', 'Name_length_Category'
    ],
    axis=1,
    inplace=True)
print(combined_train_test.describe())
# 12. 整理数据

train_data = combined_train_test[:891]
test_data = combined_train_test[891:]

titanic_train_data_X = train_data.drop(['Survived'], axis=1)
titanic_train_data_Y = train_data['Survived']

titanic_test_data_X = test_data.drop(['Survived'], axis=1)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   5 out of  10 | elapsed:    5.5s remaining:    5.5s
[Parallel(n_jobs=25)]: Done  10 out of  10 | elapsed:    5.5s finished
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Age feature Best GB Params:{'learning_rate': 0.01, 'max_depth': 3, 'max_features': 3, 'n_estimators': 2000}
Age feature Best GB Score:-112.69304020538209
GB Train Error for "Age" Feature Regressor:-91.49364932164973
5     33.521838
17    33.281971
19    33.045497
26    26.564377
Name: Age_GB, dtype: float64
Fitting 10 folds for each of 1 candidates, totalling 10 fits


/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   5 out of  10 | elapsed:    1.3s remaining:    1.3s
[Parallel(n_jobs=25)]: Done  10 out of  10 | elapsed:    1.5s finished
/Users/liwei/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:33: S

Age feature Best LR Params:{'fit_intercept': True, 'normalize': True}
Age feature Best LR Score:-1.0264141267662832e+25
LR Train Error for "Age" Feature Regressor-114.8232673040153
5     33.50
17    33.75
19    29.50
26    25.25
Name: Age_LRF, dtype: float64
shape1 (263,) (263, 2)
5     29.385068
17    29.385068
19    29.385068
26    29.385068
Name: Age, dtype: float64


/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


          Survived       Pclass           Age        SibSp        Parch  \
count  1309.000000  1309.000000  1.309000e+03  1309.000000  1309.000000   
mean      0.261268     2.294882 -2.178039e-16     0.498854     0.385027   
std       0.439494     0.837836  1.000382e+00     1.041658     0.865560   
min       0.000000     1.000000 -2.299062e+00     0.000000     0.000000   
25%       0.000000     2.000000 -6.041605e-01     0.000000     0.000000   
50%       0.000000     3.000000 -3.077692e-02     0.000000     0.000000   
75%       1.000000     3.000000  4.051716e-01     1.000000     0.000000   
max       1.000000     3.000000  3.899013e+00     8.000000     9.000000   

               Fare   Embarked_C   Embarked_Q   Embarked_S   Sex_female  \
count  1.309000e+03  1309.000000  1309.000000  1309.000000  1309.000000   
mean  -1.302752e-16     0.206264     0.093965     0.699771     0.355997   
std    1.000382e+00     0.404777     0.291891     0.458533     0.478997   
min   -8.759798e-01     

In [13]:
# 13. 利用特征值重要性排名来去除无用列
feature_to_pick = 250
feature_top_n = get_top_n_features(titanic_train_data_X, titanic_train_data_Y, feature_to_pick)
print('Total Feature:' + str(combined_train_test.shape))
print('Picked Feature' + str(feature_top_n.shape))

titanic_train_data_X = titanic_train_data_X[feature_top_n]
del titanic_train_data_X['Ticket_Number']
titanic_test_data_X = titanic_test_data_X[feature_top_n]
del titanic_test_data_X['Ticket_Number']

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  13 out of  20 | elapsed:    8.3s remaining:    4.5s
[Parallel(n_jobs=25)]: Done  20 out of  20 | elapsed:   10.9s finished


Top N Features Best RF Params:{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 500}
Top N Features Best RF Score:0.8338945005611672
Top N Features RF Train Error:0.9427609427609428
Sample 25 Features from RF Classifier
8                   Sex_female
9                     Sex_male
12                    Title_Mr
16                 Name_Length
4                         Fare
11                  Title_Miss
1                          Age
1134             Ticket_Number
1120      Pclass_Fare_Category
13                   Title_Mrs
0                       Pclass
1121               Family_Size
1116           Fare_Category_1
2                        SibSp
1124    Family_Size_Category_2
3                        Parch
18      Name_length_Category_1
7                   Embarked_S
17      Name_length_Category_0
1117           Fare_Category_2
20      Name_length_Category_3
19      Name_length_Category_2
5                   Embarked_C
1123    Family_Size_Category_1
10                Title_Mast

[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  13 out of  20 | elapsed:   18.3s remaining:    9.8s
[Parallel(n_jobs=25)]: Done  20 out of  20 | elapsed:   23.9s finished


Top N Features Best Ada Params:{'learning_rate': 0.5, 'n_estimators': 500}
Top N Features Best Ada Score:0.8473625140291807
Top N Features Ada Train Error:0.9988776655443322
Sample 25 Feature from Ada Classifier:
1134             Ticket_Number
4                         Fare
1                          Age
9                     Sex_male
8                   Sex_female
12                    Title_Mr
16                 Name_Length
10                Title_Master
1239               Ticket_1601
2106      Ticket_Letter_STON/O
37          First_Name_Allison
2193             Cabin_C22 C26
13                   Title_Mrs
14               Title_Officer
5                   Embarked_C
1120      Pclass_Fare_Category
1115           Fare_Category_0
11                  Title_Miss
2070        Ticket_Letter_A/5.
1121               Family_Size
18      Name_length_Category_1
2                        SibSp
7                   Embarked_S
409         First_Name_Jussila
1128            Age_Category_2
Name: featur

[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  13 out of  20 | elapsed:    6.1s remaining:    3.3s
[Parallel(n_jobs=25)]: Done  20 out of  20 | elapsed:    8.2s finished


Top N Features Best ET Params:{'max_depth': 15, 'min_samples_split': 3, 'n_estimators': 500}
Top N Features Best ET Score:0.8338945005611672
Top N Features ET Train Error:0.920314253647587
Sample 25 Features from ET Classifier:
9                     Sex_male
12                    Title_Mr
8                   Sex_female
13                   Title_Mrs
11                  Title_Miss
0                       Pclass
1120      Pclass_Fare_Category
1116           Fare_Category_1
1124    Family_Size_Category_2
1117           Fare_Category_2
16                 Name_Length
17      Name_length_Category_0
18      Name_length_Category_1
1123    Family_Size_Category_1
4                         Fare
20      Name_length_Category_3
7                   Embarked_S
1121               Family_Size
2304            Cabin_Letter_E
19      Name_length_Category_2
5                   Embarked_C
2301            Cabin_Letter_B
2303            Cabin_Letter_D
2                        SibSp
1127            Age_Category

In [14]:
# 14.建立模型
rf_est = ensemble.RandomForestClassifier(n_estimators=750, criterion='gini', max_features='sqrt',
                                         max_depth=3, min_samples_split=4, min_samples_leaf=2,
                                         n_jobs=50, random_state=42, verbose=1)
gbm_est = ensemble.GradientBoostingClassifier(n_estimators=900, learning_rate=0.0008, loss='exponential',
                                              min_samples_split=3, min_samples_leaf=2, max_features='sqrt',
                                              max_depth=3, random_state=42, verbose=1)
et_est = ensemble.ExtraTreesClassifier(n_estimators=750, max_features='sqrt', max_depth=35, n_jobs=50,
                                       criterion='entropy', random_state=42, verbose=1)

voting_est = ensemble.VotingClassifier(estimators=[('rf', rf_est), ('gbm', gbm_est), ('et', et_est)],
                                       voting='soft', weights=[3, 5, 2],
                                       n_jobs=50)
voting_est.fit(titanic_train_data_X, titanic_train_data_Y)
print('VotingClassifier Score:' + str(voting_est.score(titanic_train_data_X, titanic_train_data_Y)))
print('VotingClassifier Estimators:' + str(voting_est.estimators_))

[Parallel(n_jobs=50)]: Using backend ThreadingBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 350 tasks      | elapsed:    0.1s
[Parallel(n_jobs=50)]: Done 750 out of 750 | elapsed:    0.2s finished
[Parallel(n_jobs=50)]: Using backend ThreadingBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 350 tasks      | elapsed:    0.1s
[Parallel(n_jobs=50)]: Done 750 out of 750 | elapsed:    0.2s finished


VotingClassifier Score:0.9270482603815937
VotingClassifier Estimators:[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=50,
            oob_score=False, random_state=42, verbose=1, warm_start=False), GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.0008, loss='exponential', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=2, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=900,
              n_iter_no_change=None, presort='auto', random_state=42,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              

In [15]:
titanic_test_data_X.drop(['Survived'], axis=1, inplace=True)

KeyError: "['Survived'] not found in axis"

In [ ]:
# 预测
titanic_test_data_X['Survived'] = voting_est.predict(titanic_test_data_X)

submission = pd.DataFrame({'PassengerId': test_data_org.loc[:, 'PassengerId'],
                           'Survived': titanic_test_data_X.loc[:, 'Survived']})

submission.to_csv('../output/submission_result_notebook.csv', index=False, sep=',')